Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print("Training set", train_dataset.shape, train_labels.shape)
    print("Validation set", valid_dataset.shape, valid_labels.shape)
    print("Test set", test_dataset.shape, test_labels.shape)

('Training set', (250000, 28, 28), (250000,))
('Validation set', (15000, 28, 28), (15000,))
('Test set', (15000, 28, 28), (15000,))


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print("Training set", train_dataset.shape, train_labels.shape)
print("Validation set", valid_dataset.shape, valid_labels.shape)
print("Test set", test_dataset.shape, test_labels.shape)

('Training set', (250000, 28, 28, 1), (250000, 10))
('Validation set', (15000, 28, 28, 1), (15000, 10))
('Test set', (15000, 28, 28, 1), (15000, 10))


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [28]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD

In [32]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', \
                 input_shape=(train_dataset.shape[1], train_dataset.shape[2], train_dataset.shape[3])))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
#model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
#model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=112, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=56, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=28, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=10, activation='softmax'))

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3136)              0         
__________

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['accuracy'])

In [36]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(train_dataset, train_labels, epochs=10, batch_size=512, callbacks=[early_stopping], \
          validation_data=(valid_dataset, valid_labels))

Train on 250000 samples, validate on 15000 samples
Epoch 1/10
250000/250000 [==============================] - 136s - loss: 1.0462 - acc: 0.6559 - val_loss: 0.4412 - val_acc: 0.8724
Epoch 2/10
250000/250000 [==============================] - 133s - loss: 0.6288 - acc: 0.8176 - val_loss: 0.4043 - val_acc: 0.8798
Epoch 3/10
250000/250000 [==============================] - 131s - loss: 0.5683 - acc: 0.8374 - val_loss: 0.3696 - val_acc: 0.8893
Epoch 4/10
250000/250000 [==============================] - 132s - loss: 0.5392 - acc: 0.8455 - val_loss: 0.3701 - val_acc: 0.8923
Epoch 5/10
250000/250000 [==============================] - 137s - loss: 0.5271 - acc: 0.8483 - val_loss: 0.3576 - val_acc: 0.8955
Epoch 6/10
250000/250000 [==============================] - 135s - loss: 0.5161 - acc: 0.8530 - val_loss: 0.3842 - val_acc: 0.8933
Epoch 7/10
250000/250000 [==============================] - 134s - loss: 0.5133 - acc: 0.8530 - val_loss: 0.3691 - val_acc: 0.8928
Epoch 8/10
250000/250000 [======

In [38]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model.fit(train_dataset, train_labels, epochs=10, batch_size=512, callbacks=[early_stopping], \
          validation_data=(valid_dataset, valid_labels))

Train on 250000 samples, validate on 15000 samples
Epoch 1/10
250000/250000 [==============================] - 129s - loss: 0.5046 - acc: 0.8568 - val_loss: 0.3995 - val_acc: 0.8929
Epoch 2/10
250000/250000 [==============================] - 124s - loss: 0.5089 - acc: 0.8560 - val_loss: 0.3662 - val_acc: 0.9020
Epoch 3/10
250000/250000 [==============================] - 132s - loss: 0.5046 - acc: 0.8567 - val_loss: 0.3640 - val_acc: 0.8955
Epoch 4/10
250000/250000 [==============================] - 128s - loss: 0.5060 - acc: 0.8576 - val_loss: 0.3543 - val_acc: 0.8989
Epoch 5/10
250000/250000 [==============================] - 133s - loss: 0.5081 - acc: 0.8566 - val_loss: 0.3394 - val_acc: 0.9021
Epoch 6/10
250000/250000 [==============================] - 131s - loss: 0.5108 - acc: 0.8557 - val_loss: 0.3485 - val_acc: 0.9002
Epoch 7/10
250000/250000 [==============================] - 126s - loss: 0.5094 - acc: 0.8559 - val_loss: 0.3977 - val_acc: 0.8932
Epoch 8/10
250000/250000 [======

In [30]:
sgd = SGD(lr=0.1, decay=0.65)
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])
model.fit(train_dataset, train_labels, epochs=10, batch_size=512, callbacks=[early_stopping], \
          validation_data=(valid_dataset, valid_labels))

Train on 250000 samples, validate on 15000 samples
Epoch 1/10
250000/250000 [==============================] - 125s - loss: 0.4512 - acc: 0.8644 - val_loss: 0.3218 - val_acc: 0.9073
Epoch 2/10
250000/250000 [==============================] - 138s - loss: 0.4461 - acc: 0.8659 - val_loss: 0.3235 - val_acc: 0.9067
Epoch 3/10
250000/250000 [==============================] - 139s - loss: 0.4460 - acc: 0.8662 - val_loss: 0.3235 - val_acc: 0.9070
Epoch 4/10
250000/250000 [==============================] - 132s - loss: 0.4439 - acc: 0.8669 - val_loss: 0.3208 - val_acc: 0.9069
Epoch 5/10
250000/250000 [==============================] - 131s - loss: 0.4443 - acc: 0.8666 - val_loss: 0.3204 - val_acc: 0.9088
Epoch 6/10
250000/250000 [==============================] - 131s - loss: 0.4444 - acc: 0.8674 - val_loss: 0.3218 - val_acc: 0.9075
Epoch 7/10
250000/250000 [==============================] - 130s - loss: 0.4428 - acc: 0.8674 - val_loss: 0.3193 - val_acc: 0.9079
Epoch 8/10
250000/250000 [======

In [39]:
model.evaluate(test_dataset, test_labels, batch_size=512)

14848/15000 [============================>.] - ETA: 0s

[0.20021333509286246, 0.94793333355585729]

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], \
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.554713
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.892809
Minibatch accuracy: 37.5%
Validation accuracy: 52.0%
Minibatch loss at step 100: 0.758339
Minibatch accuracy: 68.8%
Validation accuracy: 70.1%
Minibatch loss at step 150: 0.879906
Minibatch accuracy: 75.0%
Validation accuracy: 75.0%
Minibatch loss at step 200: 0.221375
Minibatch accuracy: 93.8%
Validation accuracy: 73.5%
Minibatch loss at step 250: 0.619155
Minibatch accuracy: 87.5%
Validation accuracy: 75.6%
Minibatch loss at step 300: 0.842778
Minibatch accuracy: 81.2%
Validation accuracy: 79.1%
Minibatch loss at step 350: 0.450421
Minibatch accuracy: 87.5%
Validation accuracy: 77.8%
Minibatch loss at step 400: 1.282068
Minibatch accuracy: 62.5%
Validation accuracy: 80.2%
Minibatch loss at step 450: 0.925103
Minibatch accuracy: 68.8%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.433502
Minibatch accuracy: 93.8%
Validation accuracy: 80.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], \
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden3, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.920558
Minibatch accuracy: 12.5%
Validation accuracy: 10.4%
Minibatch loss at step 50: 1.837903
Minibatch accuracy: 50.0%
Validation accuracy: 57.6%
Minibatch loss at step 100: 0.712446
Minibatch accuracy: 62.5%
Validation accuracy: 66.4%
Minibatch loss at step 150: 0.795561
Minibatch accuracy: 75.0%
Validation accuracy: 75.5%
Minibatch loss at step 200: 0.302790
Minibatch accuracy: 87.5%
Validation accuracy: 75.5%
Minibatch loss at step 250: 0.465574
Minibatch accuracy: 87.5%
Validation accuracy: 78.5%
Minibatch loss at step 300: 0.619193
Minibatch accuracy: 75.0%
Validation accuracy: 80.3%
Minibatch loss at step 350: 0.401282
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 400: 1.031667
Minibatch accuracy: 68.8%
Validation accuracy: 81.2%
Minibatch loss at step 450: 0.898039
Minibatch accuracy: 68.8%
Validation accuracy: 80.5%
Minibatch loss at step 500: 0.499542
Minibatch accuracy: 93.8%
Validation accuracy: 82.7%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [9]:
batch_size = 128
patch_size = 5
depth = 16
num_hidden = 128
initial_learning_rate = 0.5
dropout_prob = 0.5

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], \
                                                     stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden1 = tf.nn.relu(conv1 + layer1_biases)
        pool1 = tf.nn.max_pool(hidden1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        # conv2 = tf.nn.conv2d(tf.nn.dropout(pool1, dropout_prob), layer2_weights, [1, 1, 1, 1], padding='SAME')
        conv2 = tf.nn.conv2d(pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden2 = tf.nn.relu(conv2 + layer2_biases)
        pool2 = tf.nn.max_pool(hidden2, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        # shape = tf.nn.dropout(pool2, dropout_prob).get_shape().as_list()
        shape = pool2.get_shape().as_list()
        reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        return tf.matmul(hidden3, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, 1000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.517567
Minibatch accuracy: 7.0%
Validation accuracy: 10.4%
Minibatch loss at step 50: 1.230909
Minibatch accuracy: 64.1%
Validation accuracy: 67.8%
Minibatch loss at step 100: 0.894895
Minibatch accuracy: 76.6%
Validation accuracy: 79.3%
Minibatch loss at step 150: 0.691989
Minibatch accuracy: 82.8%
Validation accuracy: 79.9%
Minibatch loss at step 200: 0.613899
Minibatch accuracy: 83.6%
Validation accuracy: 80.0%
Minibatch loss at step 250: 0.862593
Minibatch accuracy: 78.1%
Validation accuracy: 81.6%
Minibatch loss at step 300: 0.621440
Minibatch accuracy: 83.6%
Validation accuracy: 82.8%
Minibatch loss at step 350: 0.612352
Minibatch accuracy: 82.8%
Validation accuracy: 83.4%
Minibatch loss at step 400: 0.607493
Minibatch accuracy: 79.7%
Validation accuracy: 82.9%
Minibatch loss at step 450: 0.588815
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 500: 0.551198
Minibatch accuracy: 85.9%
Validation accuracy: 84.2%
Mi